## Intermediate Lab:  Creating A Dataset Using the GitHub API

In this lab you'll create a dataset containing all the meta information about your classmates github repos, using only the csv file with everyone's url.  

The process will be done in these 4 general steps:

 - load in the csv file
 - clean the individual lines of each file to get them ready to use
 - connect to the GitHub api to obtain information about everyone's repos
 - re-structure this information to turn it into a dataframe.
 
It'll be a great way to work through the first step of many data science problems: creating a workable dataset out of unorganized, messy data.  Let's get started!

### Step 1:  Load in the csv file with everyone's github repo

**Note:** There are a number of ways to do this, but the easiest way is usually this:

    `with open('file.csv') as f:

        data = f.readlines()`

In [1]:
with open('../GitHub Repos.csv', 'r') as f:
    lines = f.readlines()

What you should have now is a list, and each item is a string that contains the comma separated values of each cell in the row of that csv file.  

It should generically look like this:

    `['Name,Repo\n',
      'Person 1,https://github.com/username\n',
      'Person 2,https://github.com/username\n',
       ......
       ]`

Double check this is the case.

In [2]:
lines


['Name,Repo\n',
 'Jonathan Bechtel,https://github.com/JonathanBechtel\n',
 'Aoife Duna,https://github.com/aoifeduna\n',
 'Christos Simos,https://github.com/simoschr\n',
 'Brittani Kauf,https://github.com/brittanikauf\n',
 'Stanford Turner,https://github.com/sturner08\n',
 'Shelly Seroussi,https://github.com/sturner08\n',
 'Shela Wu,https://github.com/misowu\n',
 'Rishabh Chandra,https://github.com/rishabhchandra35\n',
 '"Behrang ""Brian"" Bidadi",https://github.com/brianb888\n',
 'Alec Schneider,https://github.com/Blitz33697\n',
 'Taku Takamatsu,https://github.com/taku-takamatsu\n',
 'Lindsey Gumann,https://github.com/lgumann\n',
 'David Hurley,https://github.com/davehurl\n',
 'Kina Abe,https://github.com/kinaabe57\n',
 'Elyse Chu,https://github.com/elysechu\n',
 'Michael Lawlor,https://github.com/lawlormj/sample_work\n',
 'Emily Lam,https://github.com/emilylam98\n',
 'Mike Golodner,https://github.com/mgolodner/ga_repo.git\n',
 'Siddharth Uppal,https://github.com/sid25393']

The only thing we need out of each item is the persons username, that part contained in the string at: `https://github.com/username_here`.  Everything else is junk.  

We'll need to go through a few steps to get our info down to a usable format.  

### Step 2: Remove the `\n` from each item in the list, as well as the item that contains the header info.

When you're done you should have a list that looks like this:

      `[
      'Person 1,https://github.com/username',
      'Person 2,https://github.com/username',
       ......
       ]`

**hint:** The `replace()` method for strings is probably one of the more useful options that you have.  If you want to replace something with nothing, you can simply specify `""` for that part.

In [4]:
lines = [line.rstrip() for line in lines[1:]]


### Step 3:  Separate the url in each string from everything else

When you're done you should have a new list that looks like this:

    `[
       'https://github.com/username',
       'https://github.com/username',
       ...
     ]`
     
**hint:** The `split()` method will help you out here.

In [7]:
urls = [line.split(',')[-1] for line in lines]
urls

['https://github.com/JonathanBechtel',
 'https://github.com/aoifeduna',
 'https://github.com/simoschr',
 'https://github.com/brittanikauf',
 'https://github.com/sturner08',
 'https://github.com/sturner08',
 'https://github.com/misowu',
 'https://github.com/rishabhchandra35',
 'https://github.com/brianb888',
 'https://github.com/Blitz33697',
 'https://github.com/taku-takamatsu',
 'https://github.com/lgumann',
 'https://github.com/davehurl',
 'https://github.com/kinaabe57',
 'https://github.com/elysechu',
 'https://github.com/lawlormj/sample_work',
 'https://github.com/emilylam98',
 'https://github.com/mgolodner/ga_repo.git',
 'https://github.com/sid25393']

### Step 4:  Separate the username in each string from everything else

When you're done you should have a new list that looks like this:

    `[
       'username1',
       'username2',
       ...
     ]`

In [10]:
users = [name.split('/')[3] for name in urls]
users

['JonathanBechtel',
 'aoifeduna',
 'simoschr',
 'brittanikauf',
 'sturner08',
 'sturner08',
 'misowu',
 'rishabhchandra35',
 'brianb888',
 'Blitz33697',
 'taku-takamatsu',
 'lgumann',
 'davehurl',
 'kinaabe57',
 'elysechu',
 'lawlormj',
 'emilylam98',
 'mgolodner',
 'sid25393']

### Step 5: Obtain the repo data for every single github username

The repository info for every single public account is available via the following url: `https://api.github.com/users/:the_username/repos`

So basically, `requests.get('https://api.github.com/users/:the_username/repos').json()` will return a list filled with every single repo that user has.  

When you're done, you should have a *list of lists*, with each list containing each users individual repos.  It'll look like this:

`[[{user1, repo1}, {user1, repo2}], [{user2, repo1}], [{user3, repo1}, {user3, repo2}, {user3, repo3}], .....]`

In [13]:
import requests
api_url = 'https://api.github.com/users/'
user_repos = [requests.get(api_url + user + '/repos').json() for user in users[:5]]

In [18]:
len(user_repos)

5

### Step 6: Create a 'flat' list that contains every unique repo for every single user

When you're done you should have a list that looks like this: `[{user1 repo1}, {user1 repo2}, ....{user n, repo m}]`

Ie, no nested levels like you had before.

In [22]:
for i in range(len(user_repos)):
    print(len(user_repos[i]))


25
3
1
12
4


In [28]:
# nested for loops to flatten the user_repos
# flat_repos = []
# for repos in user_repors:
#     for repo in repos:
#         flat_repos.append(repo)
flat_repos = [repo for repos in user_repos for repo in repos]

### Step 7:  Get information about the name, owner, url, and date of every single repo.

In the dictionary for each repo there are keys called `name`, `login`, `html_url`, and `created_at`.  These are going to populate the values for our different columns.

Values for each one of these keys will need to exist inside their own lists.

**hint:** Notice that the `login` key is nested inside a dictionary that's the value to the `owner` key at the outer level.

In [30]:
owners = [repo['owner']['login'] for repo in flat_repos]
names = [repo['name'] for repo in flat_repos]
urls = [repo['html_url'] for repo in flat_repos]
dates = [repo['created_at'] for repo in flat_repos]
owners

['JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'JonathanBechtel',
 'aoifeduna',
 'aoifeduna',
 'aoifeduna',
 'simoschr',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'brittanikauf',
 'sturner08',
 'sturner08',
 'sturner08',
 'sturner08']

### Step 8:  Create a dictionary with the data created from step 7

Your final output should look like this:

    `{
       'Owner': [list with the `login` values],
       'Name' : [list with the `name` values],
       'URL'  : [list with the `html_url` values],
       'Date' : [list with the `created_at` values]
     }`

In [32]:
data_dict = {
    'Owner': owners,
    'Name': names,
    'URL': urls,
    'Date': dates
}
data_dict

{'Owner': ['JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'JonathanBechtel',
  'aoifeduna',
  'aoifeduna',
  'aoifeduna',
  'simoschr',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'brittanikauf',
  'sturner08',
  'sturner08',
  'sturner08',
  'sturner08'],
 'Name': ['cdc-dashboard',
  'DAT-01-21',
  'DAT-06-24',
  'DAT-10-14',
  'data',
  'Data-Analysis',
  'easyml',
  'formula-app',


### Step 9:  Pass your dictionary into the `pd.dataframe()` method to get your final dataset  

Use the `df.head()` method to confirm that you have something that's formatted appropriately.

In [34]:
import pandas as pd
df = pd.DataFrame(data_dict)
df.head()

,Owner,Name,URL,Date
0,JonathanBechtel,cdc-dashboard,https://github.com/JonathanBechtel/cdc-dashboard,2016-11-02T14:39:37Z
1,JonathanBechtel,DAT-01-21,https://github.com/JonathanBechtel/DAT-01-21,2020-01-21T12:57:43Z
2,JonathanBechtel,DAT-06-24,https://github.com/JonathanBechtel/DAT-06-24,2019-06-26T15:12:49Z
3,JonathanBechtel,DAT-10-14,https://github.com/JonathanBechtel/DAT-10-14,2019-10-14T16:13:47Z
4,JonathanBechtel,data,https://github.com/JonathanBechtel/data,2019-01-14T22:09:06Z
